In [5]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 848.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [7]:
import re
from typing import Dict, List, Optional, Tuple
from datetime import datetime
import logging

# Mock external libraries (replace with real integrations in production)
class MockLLM:
    def classify(self, text: str) -> Tuple[str, float]:
        """Mock classification with similarity score (0.0 to 1.0)."""
        text_lower = text.lower()
        categories = {
            "Loan Application - Urgent": ["urgent", "loan", "process"],
            "Fraud Report": ["fraud", "suspect", "account"],
            "Customer Inquiry": ["question", "help", "inquiry"]
        }

        # Simple mock similarity scoring based on keyword overlap
        best_category = "Customer Inquiry"
        best_score = 0.1  # Default low score

        for category, keywords in categories.items():
            score = sum(1 for kw in keywords if kw in text_lower) / len(keywords)
            if "urgent" in text_lower and "loan" in text_lower:
                score += 0.3  # Boost for urgent loans
            score = min(1.0, max(0.0, score))  # Normalize to 0-1
            if score > best_score:
                best_score = score
                best_category = category

        return best_category, best_score

    def extract_entities(self, text: str) -> Dict[str, str]:
        """Mock entity extraction."""
        entities = {}
        account_match = re.search(r"Account #(\d+)", text)
        name_match = re.search(r"Name: (\w+ \w+)", text)
        amount_match = re.search(r"\$(\d+)", text)
        if account_match:
            entities["account_number"] = account_match.group(1)
        if name_match:
            entities["name"] = name_match.group(1)
        if amount_match:
            entities["amount"] = amount_match.group(1)
        return entities

class MockEmailClient:
    def send_email(self, to: str, subject: str, body: str):
        print(f"Sending email to {to}: Subject: {subject}, Body: {body}")

class MockCRM:
    def get_client_status(self, account_number: str) -> str:
        return "VIP" if account_number == "12345" else "Regular"

# Configure logging
logging.basicConfig(filename="triage_log.log", level=logging.INFO,
                    format="%(asctime)s - %(message)s")

class GenAIOrchestrator:
    def __init__(self):
        self.llm = MockLLM()
        self.email_client = MockEmailClient()
        self.crm = MockCRM()
        self.routing_rules = {
            "Loan Application - Urgent": {"dept": "Loan Processing", "escalate": True},
            "Fraud Report": {"dept": "Fraud Team", "escalate": True},
            "Customer Inquiry": {"dept": "Customer Service", "escalate": False},
        }
        self.similarity_threshold = 0.5  # Minimum score to trust classification

    def preprocess(self, content: str, attachment: Optional[str] = None) -> str:
        """Combine email body and attachment text."""
        processed_text = content
        if attachment:
            processed_text += f" {attachment}"
        return processed_text.strip()

    def classify_and_extract(self, text: str) -> Tuple[str, float, Dict[str, str]]:
        """Classify content with similarity score and extract entities."""
        category, similarity_score = self.llm.classify(text)
        entities = self.llm.extract_entities(text)
        return category, similarity_score, entities

    def determine_priority(self, category: str, entities: Dict[str, str]) -> str:
        """Determine priority based on category and client status."""
        account_number = entities.get("account_number", "")
        client_status = self.crm.get_client_status(account_number)
        if "Urgent" in category or client_status == "VIP":
            return "High"
        return "Normal"

    def route(self, category: str, similarity_score: float, priority: str,
              entities: Dict[str, str]) -> Dict[str, str]:
        """Route to appropriate department with similarity score check."""
        if similarity_score < self.similarity_threshold:
            routing_info = {"dept": "Manual Review", "escalate": False}
        else:
            routing_info = self.routing_rules.get(category, {"dept": "General", "escalate": False})

        dept = routing_info["dept"]
        escalate = routing_info["escalate"] and priority == "High"

        routing_decision = {
            "department": dept,
            "escalate_to": "Manager" if escalate else None,
            "assigned_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "similarity_score": similarity_score
        }
        return routing_decision

    def log_action(self, email_id: str, category: str, similarity_score: float,
                   entities: Dict[str, str], routing_decision: Dict[str, str]):
        """Log the triage and routing decision with similarity score."""
        log_message = (f"Email ID: {email_id}, Category: {category}, "
                       f"Similarity Score: {similarity_score:.2f}, Entities: {entities}, "
                       f"Routed to: {routing_decision['department']}, "
                       f"Escalated: {routing_decision['escalate_to']}")
        logging.info(log_message)

    def send_auto_reply(self, sender: str, category: str, similarity_score: float):
        """Send an acknowledgment email, with caution if similarity is low."""
        subject = "Request Received"
        if similarity_score < self.similarity_threshold:
            body = "Thank you! Your request requires manual review and will be processed soon."
        else:
            body = f"Thank you! Your {category} request is being processed."
        self.email_client.send_email(sender, subject, body)

    def process_email(self, email_id: str, sender: str, content: str,
                      attachment: Optional[str] = None):
        """Main orchestration function with similarity score."""
        # Preprocess
        text = self.preprocess(content, attachment)

        # Classify and extract entities
        category, similarity_score, entities = self.classify_and_extract(text)

        # Determine priority
        priority = self.determine_priority(category, entities)

        # Route
        routing_decision = self.route(category, similarity_score, priority, entities)

        # Log the action
        self.log_action(email_id, category, similarity_score, entities, routing_decision)

        # Send auto-reply
        self.send_auto_reply(sender, category, similarity_score)

        # Return result for debugging/demo
        return {
            "category": category,
            "similarity_score": similarity_score,
            "entities": entities,
            "priority": priority,
            "routing": routing_decision
        }

# Test the orchestrator
if __name__ == "__main__":
    orchestrator = GenAIOrchestrator()

    # Test Case 1: Urgent Loan Application
    email_id = "E123"
    sender = "john.doe@example.com"
    content = "Urgent: Please process my loan. Account #12345"
    attachment = "Name: John Doe, Amount: $50000"

    result = orchestrator.process_email(email_id, sender, content, attachment)
    print("Test Case 1 Result:", result)

    # Test Case 2: Fraud Report
    email_id = "E124"
    sender = "jane.smith@example.com"
    content = "I suspect fraud on my account #67890"

    result = orchestrator.process_email(email_id, sender, content)
    print("Test Case 2 Result:", result)

    # Test Case 3: Ambiguous Email (low similarity)
    email_id = "E125"
    sender = "bob@example.com"
    content = "Hello, I need something done."

    result = orchestrator.process_email(email_id, sender, content)
    print("Test Case 3 Result:", result)

Sending email to john.doe@example.com: Subject: Request Received, Body: Thank you! Your Loan Application - Urgent request is being processed.
Test Case 1 Result: {'category': 'Loan Application - Urgent', 'similarity_score': 1.0, 'entities': {'account_number': '12345', 'name': 'John Doe', 'amount': '50000'}, 'priority': 'High', 'routing': {'department': 'Loan Processing', 'escalate_to': 'Manager', 'assigned_at': '2025-03-25 16:10:08', 'similarity_score': 1.0}}
Sending email to jane.smith@example.com: Subject: Request Received, Body: Thank you! Your Fraud Report request is being processed.
Test Case 2 Result: {'category': 'Fraud Report', 'similarity_score': 1.0, 'entities': {}, 'priority': 'Normal', 'routing': {'department': 'Fraud Team', 'escalate_to': None, 'assigned_at': '2025-03-25 16:10:08', 'similarity_score': 1.0}}
Sending email to bob@example.com: Subject: Request Received, Body: Thank you! Your request requires manual review and will be processed soon.
Test Case 3 Result: {'cate